In [1]:
from sentence_transformers import SentenceTransformer
import os
import numpy as np
from pathlib import Path
import time
import pandas as pd

In [ ]:
os.listdir(Path("/home/max/Models/sentenceBERT"))

In [6]:
def naming(full_path):
    return full_path.split("/")[-1]

In [ ]:
os.listdir("/home/max/Results/replacements/data/") + []

In [2]:
df = pd.read_csv("/home/max/Results/replacements/data/aterinvandring/ingroup/first_round/replacements.txt", sep="\t", index_col = 0)

In [3]:
df.iloc[:,0]

0       Repatriering, kompensatorisk migration, deport...
3                        Repatriering, tvångsförflyttning
10                                           Återvändande
12                                           Återvändande
25                                Att kasta ut invandrare
                              ...                        
3172                                            Utvisning
3209                                            Utvisning
3217                                            utvisning
3225                                            Utvisning
3241                                            Utvisning
Name: aterinvandring_text_w1, Length: 211, dtype: object

In [5]:
len(df.loc[:52])

9

In [32]:
def text2bert(data_path, local_models_at, huggface_models = []):
    
    t0 = time.time()
    
    data_path = Path(data_path)
    models_path = Path(local_models_at)
    
    models = [f"{local_models_at}/{model}" for model in os.listdir(local_models_at)] + huggface_models
    #models = os.listdir(local_models_at) + huggface_models
    
    for dwe in os.listdir(data_path):
        for meaning in ["ingroup", "outgroup"]:
            for rnd in ["first_round", "second_round"]:
                isExist = os.path.exists(data_path / dwe / meaning / rnd / "vectors")
                if not isExist:
                    os.makedirs(data_path / dwe / meaning / rnd / "vectors")   
                
                replacements = pd.read_csv(data_path / dwe / meaning / rnd / "replacements.txt", sep = "\t", index_col = 0) 
                
                # preprocessing ...
                # lower
                
                for model in models:
                    t = time.time()
                    path = data_path / dwe / meaning / rnd / "vectors" / naming(model)
                    
                    isExist = os.path.exists(path)
                    if not isExist:
                        os.makedirs(path)   
                    print()
                    print(f"{dwe:<15}{meaning:<10}{rnd:<15}{naming(model)}")
                    sBERT = SentenceTransformer(model)
                    vectors = []
                    for idx, line in zip(replacements.index, replacements.iloc[:,0]):
                        pcent = round((len(replacements.loc[:idx]) / len(replacements)) * 100, 1)
                        print(f"{pcent:<10}{int((time.time()-t))} s.", end="\r")
                        vector = sBERT.encode(line)
                        as_str = " ".join([str(value) for value in vector.tolist()])
                        vectors.append(f"{idx}\t{as_str}\n")
                    
                    with open(path / "vecs.txt", mode = "w") as f:
                        for vec in vectors:
                            f.write(vec)
    
    print()
    print("Done!", int((time.time()-t0)/60), "m.")

In [8]:
where_my_models_are = "/home/max/Models/sentenceBERT"

In [33]:
text2bert(
    data_path = Path("/home/max/Results/replacements/data/"), 
    local_models_at = where_my_models_are, 
    huggface_models = ["KBLab/sentence-bert-swedish-cased"], 
)


aterinvandring ingroup   first_round    sts_fbmodel_big_40epochs
100.0     0 m.
aterinvandring ingroup   first_round    fb_nli
100.0     0 m.
aterinvandring ingroup   first_round    sts_fbmodel
100.0     0 m.
aterinvandring ingroup   first_round    sentence-bert-swedish-cased
100.0     1 m.
aterinvandring ingroup   second_round   sts_fbmodel_big_40epochs
100.0     2 m.
aterinvandring ingroup   second_round   fb_nli
100.0     2 m.
aterinvandring ingroup   second_round   sts_fbmodel
100.0     3 m.
aterinvandring ingroup   second_round   sentence-bert-swedish-cased
100.0     4 m.
aterinvandring outgroup  first_round    sts_fbmodel_big_40epochs
100.0     5 m.
aterinvandring outgroup  first_round    fb_nli
100.0     5 m.
aterinvandring outgroup  first_round    sts_fbmodel
100.0     6 m.
aterinvandring outgroup  first_round    sentence-bert-swedish-cased
100.0     7 m.
aterinvandring outgroup  second_round   sts_fbmodel_big_40epochs
100.0     9 m.
aterinvandring outgroup  second_round   fb_